In [62]:
import numpy as np
from scipy import stats
from numpy.linalg import inv
from sigmoid import sigmoid

In [75]:
y = 1.0 * np.random.choice(2, size = 200)
mvn = stats.multivariate_normal.rvs
X = np.array([mvn([1, 1] if heads else [4, 4]) for heads in y])

In [79]:
N, D = X.shape
w = np.zeros(D)
ybar = np.mean(y)
w0 = np.log(ybar / (1 - ybar))

for k in range(2):
    eta = w0 + X.dot(w)
    mu = sigmoid(eta)
    s = mu * (1 - mu)
    z = eta + (y - mu) / s
    S = np.diag(s)
    w = inv(X.T.dot(S).dot(X)).dot(X.T).dot(z)